DATA TRANSFORMATION

Objectives
1. Divide data by teams
2. Combine rows so that one row is describing one game
3. Shift the result column in time so that the data in one row predicts the outcome of the next game

In [2]:
import pandas as pd
import numpy as np

In the cell below I take advantege of the structure eof the dataset. If the team is blue side then their oponents are going to be in the row i+1, if the team is red side their oponents are going to be in the row i-1

In [3]:
df = pd.read_csv("teams_data.csv")
teams = df.teamname.unique()
for team in teams:
    list = []
    for i in range(len(df)-1):
        if df.loc[i, "teamname"] == team and df.loc[i, "side"] == "Blue":
            list.append(df.loc[i, :])
            list.append(df.loc[i+1, :])
        elif df.loc[i, "teamname"] == team and df.loc[i, "side"] == "Red":
            list.append(df.loc[i, :])
            list.append(df.loc[i-1, :])
    data_ready_2 = pd.DataFrame(list)
    data_ready_2.to_csv(f"data_ready_{team}.csv")

Now, according to my assumtions, I create dataframes that contain mean scores of how the particular team faired in past matches and the future results

In [4]:
df1 = pd.read_csv("teams_data.csv")
teams = df1.teamname.unique()
for team in teams:
    df = pd.read_csv(f"data_ready_{team}.csv")
    model_stats = ["team kpm", "dragons", "barons", "towers", "dpm", "vspm", "earned gpm", 'cspm', 'xpat15', 'goldat15']
    df['oponents'] = 0
    for i in range(len(df)):
        if df.loc[i, "teamname"] == team:
            df.loc[i, model_stats] = df.loc[i, model_stats] - df.loc[i+1, model_stats]
        if df.loc[i, "teamname"] != team:
            df.loc[i-1, 'oponents'] = df.loc[i, 'teamname']
    df = df[df['teamname'] == team]
    df_means = df[["teamname", "team kpm", "dragons", "barons", "towers", "dpm", "vspm", "earned gpm", 'cspm', 'xpat15', 'goldat15', 'result', 'oponents']]
    new_cols = ["team kpm means", "dragons means", "barons means", "towers means", "dpm means", "vspm means", "earned gpm means", 'cspm means', 
            'xpat15 means', 'goldat15 means', 'future result']
    df_means[new_cols] = 0
    means_indexes = ["team kpm means", "dragons means", "barons means", "towers means", "dpm means", 
                 "vspm means", "earned gpm means", 'cspm means', 'xpat15 means', 'goldat15 means']
    df_means.reset_index(inplace=True, drop=True)
    for j in range(len(means_indexes)):
        for i in range(1, len(df_means)):
            x = np.mean(df_means.iloc[:i, j+1])
            df_means.loc[i-1, means_indexes[j]] = x
    for i in range(len(df_means)):
        result_future = df_means.loc[i, "result"]
        df_means.loc[i+1, "future result"] = result_future
    df_means = df_means.drop(df_means.index[-1])
    df_means = df_means.drop(df_means.index[-1])
    df_means = df_means[["team kpm means", "dragons means", "barons means", "towers means", "dpm means", 
                 "vspm means", "earned gpm means", 'cspm means', 'xpat15 means', 'goldat15 means', 'future result', 'oponents']]
    df_means.to_csv(f"mean_stats_{team}.csv")

C:\Users\Jakub\AppData\Local\Temp\ipykernel_22212\3322010126.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Rare Atom' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[i-1, 'oponents'] = df.loc[i, 'teamname']
C:\Users\Jakub\AppData\Local\Temp\ipykernel_22212\3322010126.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_means[new_cols] = 0
C:\Users\Jakub\AppData\Local\Temp\ipykernel_22212\3322010126.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

Now let's head to the second part where I combine the results from above to the actuall X and Y matrices that can be used for modeling